# Script for showing predictions

## Imports

In [2]:
#pip install opencv-python

In [32]:
import os
from shutil import rmtree
import numpy as np
import matplotlib.pyplot as plt

import json
import cv2
import matplotlib.patches as patches
import albumentations as alb

import glob
from natsort import natsorted
import cv2

## Loading predictions from file

In [37]:
PREDICTED_IMAGES_PATH = "/mnt/sds-hd/sd22a004/guest/object_detection/tmp_images/"
DATA_PATH = "/mnt/sds-hd/sd22a004/guest/data_preprocessed/data_testset/"
PREDICTIONS_PATH = "/mnt/sds-hd/sd22a004/guest/predictions/"

VIDEO_PATH = "/mnt/sds-hd/sd22a004/guest/object_detection"
VIDEO_NAME = "video_mono_left"
FPS = 20
SIDE_NUM = {'l': '02', 'r': '03'}

f = open(os.path.join(PREDICTIONS_PATH, 'predictions_mono_100epochs_test.json'), 'r')
json_decode = json.load(f)

filenames = json_decode['filenames']
predictions = json_decode['predictions']

## Define helper functions

In [38]:
def extract_info(index):
    file_args = filenames[idx].split()
    annos_pred = predictions[idx]
    return file_args, annos_pred

def load_image(file_args):
    image_path = os.path.join(DATA_PATH, file_args[0], f'image_{SIDE_NUM[file_args[2]]}', 'images', file_args[1] + '.png')
    img = cv2.imread(image_path)
    return img

def resize_boxes(original_image, annos_pred):
    # Calculate Bounding boxes back to original size
    original_height, original_width, channels = original_image.shape
    img_small = cv2.resize(original_image, (256, 256))

    aug = alb.Compose([alb.Resize(height=original_height, width=original_width)], bbox_params = alb.BboxParams(format = 'pascal_voc', label_fields = ["class_labels"]))

    augmented_pred = aug(image=np.asarray(img_small), bboxes=annos_pred['boxes'], class_labels=annos_pred['labels'])
    annos_pred = {'boxes': augmented_pred['bboxes'], 'labels': augmented_pred['class_labels']}
    
    return annos_pred

## Create images

In [ ]:
COLORS = ['red', 'blue', 'green', 'yellow', 'magenta', 'cyan']

if not os.path.exists(PREDICTED_IMAGES_PATH):
    os.mkdir(PREDICTED_IMAGES_PATH)

for idx in range(0, len(filenames)):
    file_args, annos_pred = extract_info(idx)
    print(f"\rProgress: {idx+1}/{len(filenames)}", end="")
    img = load_image(file_args)
    annos_pred = resize_boxes(img, annos_pred)
    
    fig = plt.figure(figsize=(30, 15))
    ax = plt.gca()
    
    # Show image with predictions
    for box, label in zip(annos_pred['boxes'], annos_pred['labels']):
        bb = patches.Rectangle((box[0], box[1]), box[2] - box[0], box[3] - box[1], linewidth=2, edgecolor=COLORS[int(label)], facecolor="none")
        ax.add_patch(bb)
        plt.text(box[0], box[1], int(label), fontsize=15, color=COLORS[int(label)])   
        ax.add_patch(bb)

    ax.imshow(img)
    plt.axis('off')
    fig.savefig(os.path.join(PREDICTED_IMAGES_PATH, f'{idx:04d}.jpg'), bbox_inches='tight', dpi=150)
    plt.close(fig)

## Create video

In [ ]:
img_array = []
i = 0
filenames = natsorted((glob.glob(os.path.join(PREDICTED_IMAGES_PATH, '*.jpg'))))

for idx, filename in enumerate(filenames):
    if (idx % 2) == 0:
        img = cv2.imread(filename)
        height, width, layers = img.shape
        size = (width,height)
        img_array.append(img)

out = cv2.VideoWriter(os.path.join(VIDEO_PATH, VIDEO_NAME + '.avi'), cv2.VideoWriter_fourcc(*'DIVX'), FPS, size)

for i in range(len(img_array)):
    out.write(img_array[i])
out.release()

rmtree(PREDICTED_IMAGES_PATH)